In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load the dataset
df = pd.read_csv('dataset.csv')

# Sample data preprocessing (you may want to add more depending on your specific needs)
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['background'] + ' ' + df['question'] + ' ' + df['answer'])

# Tokenize and pad sequences
def tokenize_and_pad(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')
    return padded

background_padded = tokenize_and_pad(df['background'])
question_padded = tokenize_and_pad(df['question'])
answer_padded = tokenize_and_pad(df['answer'])

# Split the data
X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(background_padded, question_padded, answer_padded, test_size=0.2)


In [ ]:
def transformer_encoder(inputs, head_size, num_heads):
    # Normalization and Attention
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=0.4)(x, x)

    # Add Skip Connection
    x = tf.keras.layers.Add()([x, inputs])

    # Feed Forward Part
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x = tf.keras.layers.Conv1D(filters=4 * head_size, kernel_size=1, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Conv1D(filters=head_size, kernel_size=1)(x)
    return tf.keras.layers.Add()([x, inputs])

# Input Layers
input_background = tf.keras.layers.Input(shape=(100,))
input_question = tf.keras.layers.Input(shape=(100,))

# Shared Embedding Layer
shared_embedding = tf.keras.layers.Embedding(input_dim=5000, output_dim=256)

# Background and Question Embedding
bg_embedding = shared_embedding(input_background)
q_embedding = shared_embedding(input_question)

# Transformer encoder
encoded_bg = transformer_encoder(bg_embedding, head_size=256, num_heads=4)
encoded_q = transformer_encoder(q_embedding, head_size=256, num_heads=4)

# Concatenate and Final Layers
concat = tf.keras.layers.Concatenate()([encoded_bg, encoded_q])
output = tf.keras.layers.Dense(100, activation='softmax')(concat)

model = tf.keras.Model(inputs=[input_background, input_question], outputs=output)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit([X1_train, X2_train], y_train, validation_data=([X1_val, X2_val], y_val), epochs=10, batch_size=64)


In [ ]:
def chat(background, question):
    # Tokenize and pad background and question
    bg_padded = tokenize_and_pad([background])
    q_padded = tokenize_and_pad([question])

    # Model prediction
    predicted = model.predict([bg_padded, q_padded])

    # Decode the predicted sequence back to text
    answer = decode_predicted_sequence(predicted)
    return answer
